In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
df = pd.read_csv("../data/curated/data.csv")
df.head()

,url,postcode,longitude,latitude,price,bed,bath,parking,desc,cbd_dist
0,https://www.domain.com.au/9-21-rockley-road-so...,3141,144.999757,-37.837338,400.0,1,1,1.0,"LIGHT, LOCATION AND IMPRESSIVE ENHANCEMENTS In...",5.9614
1,https://www.domain.com.au/35-233-canterbury-ro...,3182,144.976195,-37.859150,360.0,1,1,1.0,PLEASE NOTE THAT THIS PROPERTY IS OFFERED UN-F...,4.1882
2,https://www.domain.com.au/2-31-howitt-street-s...,3141,145.002817,-37.846776,320.0,1,1,NaN,ARRANGE AN INSPECTION TIME ONLINE - DETAILS BE...,6.9114
3,https://www.domain.com.au/2-17-railway-parade-...,3163,145.069344,-37.891722,360.0,1,1,1.0,The Vida apartments offer quality living in a ...,14.3864
4,https://www.domain.com.au/4-653-park-street-br...,3056,144.964991,-37.778276,380.0,1,1,1.0,AFFORDABLE OPPORTUNITY IN EXCLUSIVE BRUNSWICK!...,8.7863


In [3]:
school = pd.read_csv("../data/raw/school-locations2022.csv", encoding = 'cp1252')

In [4]:
sub = school[['School_Name', 'Postal_Postcode']]  # select specific columns for further process
sub = sub.rename(columns = {'Postal_Postcode': 'postcode'})
sub.head()

,School_Name,postcode
0,Alberton Primary School,3971
1,Allansford and District Primary School,3277
2,Avoca Primary School,3467
3,Avenel Primary School,3664
4,Warrandyte Primary School,3113


In [5]:
sub = sub.groupby('postcode').count()

In [6]:
sub

,School_Name
postcode,
3000,4
3003,1
3004,4
3006,2
3008,1
...,...
3987,1
3988,1
3992,1


In [7]:
sub.isnull().sum() # check nan value

School_Name    0
dtype: int64

In [8]:
# merge school with rental properties on postcode
all_df = pd.merge(df, sub, on = 'postcode', how = 'inner') 


In [10]:
all_df = all_df.rename(columns = {'School_Name': 'school_counts'})

In [11]:
all_df

,url,postcode,longitude,latitude,price,bed,bath,parking,desc,cbd_dist,school_counts
0,https://www.domain.com.au/9-21-rockley-road-so...,3141,144.999757,-37.837338,400.0,1,1,1.0,"LIGHT, LOCATION AND IMPRESSIVE ENHANCEMENTS In...",5.9614,6
1,https://www.domain.com.au/2-31-howitt-street-s...,3141,145.002817,-37.846776,320.0,1,1,NaN,ARRANGE AN INSPECTION TIME ONLINE - DETAILS BE...,6.9114,6
2,https://www.domain.com.au/505-79-river-street-...,3141,144.997588,-37.836336,470.0,1,1,1.0,CONTEMPORARY STYLE IN A PRIME POSITIONWith its...,5.8501,6
3,https://www.domain.com.au/24-43-caroline-stree...,3141,144.987560,-37.837786,395.0,1,1,1.0,ARRANGE AN INSPECTION TIME ONLINE - DETAILS BE...,4.7531,6
4,https://www.domain.com.au/1-16-18-millswyn-str...,3141,144.980266,-37.837051,475.0,1,1,NaN,Larger than average art deco style one bedroom...,3.9950,6
...,...,...,...,...,...,...,...,...,...,...,...
3953,https://www.domain.com.au/3-9-el-centro-chirns...,3116,145.312815,-37.755385,390.0,2,1,1.0,This two-bedroom apartment in the heart of Chi...,41.7823,1
3954,https://www.domain.com.au/7-14-william-street-...,3913,145.186943,-38.259360,340.0,2,1,NaN,Nestled in a quiet complex in Tyabb and within...,69.8220,3
3955,https://www.domain.com.au/403-93-furlong-road-...,3023,144.790348,-37.757284,400.0,2,2,1.0,Positioned in the heart of Cairnlea is this im...,22.6305,16
3956,https://www.domain.com.au/26a-forrest-street-s...,3429,144.710552,-37.579872,260.0,2,1,1.0,Neat and Cosy 2 bedroom Unit tucked away behin...,45.0515,14


In [12]:
# find the index of rental properties which are furnished
furnish_idx = all_df.index[all_df['desc'].str.contains('furnish|Furnish').fillna(False)] 
furnish_idx


Int64Index([  23,   65,   96,  104,  118,  147,  181,  192,  199,  205,
            ...
            3838, 3840, 3843, 3854, 3855, 3858, 3859, 3860, 3861, 3933],
           dtype='int64', length=391)

In [13]:
furnish_lst = []
for i in range(len(all_df)):
    furnish_lst.append(0)   ## 0 represents not furnished
for i in range(0, len(furnish_lst)):
    if i in furnish_idx:
        furnish_lst[i] = 1   ## 1 represents furnished


In [14]:
all_df['furnished'] = furnish_lst
all_df.head()


,url,postcode,longitude,latitude,price,bed,bath,parking,desc,cbd_dist,school_counts,furnished
0,https://www.domain.com.au/9-21-rockley-road-so...,3141,144.999757,-37.837338,400.0,1,1,1.0,"LIGHT, LOCATION AND IMPRESSIVE ENHANCEMENTS In...",5.9614,6,0
1,https://www.domain.com.au/2-31-howitt-street-s...,3141,145.002817,-37.846776,320.0,1,1,NaN,ARRANGE AN INSPECTION TIME ONLINE - DETAILS BE...,6.9114,6,0
2,https://www.domain.com.au/505-79-river-street-...,3141,144.997588,-37.836336,470.0,1,1,1.0,CONTEMPORARY STYLE IN A PRIME POSITIONWith its...,5.8501,6,0
3,https://www.domain.com.au/24-43-caroline-stree...,3141,144.987560,-37.837786,395.0,1,1,1.0,ARRANGE AN INSPECTION TIME ONLINE - DETAILS BE...,4.7531,6,0
4,https://www.domain.com.au/1-16-18-millswyn-str...,3141,144.980266,-37.837051,475.0,1,1,NaN,Larger than average art deco style one bedroom...,3.9950,6,0


In [15]:
all_df = all_df[all_df["price"] < 5000]
all_df = all_df.drop(columns = ['desc'])
all_df.head()

,url,postcode,longitude,latitude,price,bed,bath,parking,cbd_dist,school_counts,furnished
0,https://www.domain.com.au/9-21-rockley-road-so...,3141,144.999757,-37.837338,400.0,1,1,1.0,5.9614,6,0
1,https://www.domain.com.au/2-31-howitt-street-s...,3141,145.002817,-37.846776,320.0,1,1,NaN,6.9114,6,0
2,https://www.domain.com.au/505-79-river-street-...,3141,144.997588,-37.836336,470.0,1,1,1.0,5.8501,6,0
3,https://www.domain.com.au/24-43-caroline-stree...,3141,144.987560,-37.837786,395.0,1,1,1.0,4.7531,6,0
4,https://www.domain.com.au/1-16-18-millswyn-str...,3141,144.980266,-37.837051,475.0,1,1,NaN,3.9950,6,0


In [16]:
all_df.to_csv(r'../data/curated/processed_data.csv', index = False)
